In [3]:
!pip install pdfplumber
!pip install camelot-py[cv] 
!pip install opencv-python 
!pip install pytesseract 
!pip install PyMuPDF
!pip install pdf2image 
!pip install PyPDF2
!pip install easyocr
!pip install tabulate

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached easyocr-1.7.2-py3-none-any.whl.metadata (10 kB)
  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached easyocr-1.7.2-py3-none-any.whl (2.9 MB)
Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable


In [30]:
#pdf_path = "testTablesMixdocx.pdf" 
pdf_path = "testTablesMixExported.pdf" 

In [32]:
# import pdfplumber

# pdf_path = "testTablesMixdocx.pdf"

# # Open the PDF
# with pdfplumber.open(pdf_path) as pdf:
#     for page_num, page in enumerate(pdf.pages):
#         # Extract text from the page
#         text = page.extract_text()
        
#         # Extract images from the page
#         images = page.images
        
#         # Check if the page has text or images
#         print(f"Page {page_num+1}:")
        
#         if text:
#             print("This page contains text.")
#         else:
#             print("This page does not contain text.")
        
#         if images:
#             print(f"This page contains {len(images)} images.")
#         else:
#             print("This page does not contain images.")
#         print("\n")


In [58]:
import pdfplumber
import easyocr
from pdf2image import convert_from_path
import numpy as np
from tabulate import tabulate  # For formatting tables

def extract_text_based_tables_with_plumber(pdf_path):
    """Extract text-based tables using pdfplumber."""
    text_tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, 1):
            for table in page.extract_tables():
                formatted_table = {"page": page_num, "table": table}
                text_tables.append(formatted_table)
    return text_tables

def extract_image_based_tables_with_easyocr(pdf_path):
    """Extract text from image-based tables using EasyOCR."""
    reader = easyocr.Reader(['en'])  # Specify language(s)
    images = convert_from_path(pdf_path)  # Convert PDF to images
    extracted_tables = []
    
    for page_num, img in enumerate(images, 1):
        # Convert image to numpy array
        img_np = np.array(img)
        # Use EasyOCR to extract text from the image
        result = reader.readtext(img_np, detail=0)
        formatted_table = {"page": page_num, "table": result}
        extracted_tables.append(formatted_table)
    
    return extracted_tables

# def extract_non_table_text_with_plumber(pdf_path):
#     """Extract non-table text from PDF using pdfplumber."""
#     non_table_text = []
#     with pdfplumber.open(pdf_path) as pdf:
#         for page_num, page in enumerate(pdf.pages, 1):
#             text = page.extract_text()
#             if text:
#                 formatted_text = {"page": page_num, "text": text.strip()}
#                 non_table_text.append(formatted_text)
#     return non_table_text

def extract_non_table_text_with_plumber(pdf_path):
    """Extract non-table text from PDF using pdfplumber, excluding table text."""
    non_table_text = []
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, 1):
            # Get all the tables on the page
            tables = page.extract_tables()
            # Get the text from the page
            text = page.extract_text() or ""
            # Create a list to keep track of areas covered by tables
            table_areas = []

            for table in tables:
                if table and len(table) > 1:  # Ensure the table is not empty and has at least two rows
                    # Get the bounding box from the first cell's coordinates
                    try:
                        # Convert coordinates to float to avoid type issues
                        x0 = float(table[0][0])
                        top = float(table[0][1])  # Use the top value from the first cell
                        x1 = float(table[0][2])
                        bottom = float(table[1][1])  # Use the top value from the second row's first cell
                        table_areas.append((x0, top, x1, bottom))
                    except (ValueError, IndexError) as e:
                        print(f"Error processing table on page {page_num}: {e}")

            #print("table_areas==", table_areas)
            # Filter out table text from the page text
            filtered_text = []
            if text:
                words = page.extract_words()
                for word in words:
                    # Get the bounding box for the word
                    x0, top, x1, bottom = word['x0'], word['top'], word['x1'], word['bottom']
                    # Check if the word falls within any of the table areas
                    if not any(
                        x0 >= area[0] and x1 <= area[2] and top >= area[1] and bottom <= area[3]
                        for area in table_areas
                    ):
                        filtered_text.append(word['text'])
                
                non_table_text_combined = " ".join(filtered_text)
                formatted_text = {"page": page_num, "text": non_table_text_combined.strip()}
                non_table_text.append(formatted_text)
    return non_table_text


    
def process_pdf(pdf_path):
    """Main function to process the PDF and extract tables and non-table text."""
    # 1. Extract text-based tables using pdfplumber
    print("Extracting text-based tables...")
    text_based_tables = extract_text_based_tables_with_plumber(pdf_path)

    # 2. Extract image-based tables using EasyOCR
    print("Extracting image-based tables...")
    image_based_tables = extract_image_based_tables_with_easyocr(pdf_path)

    # 3. Extract non-table text using pdfplumber
    print("Extracting non-table text...")
    non_table_text = extract_non_table_text_with_plumber(pdf_path)

    return {
        "text_based_tables": text_based_tables,
        "image_based_tables": image_based_tables,
        "non_table_text": non_table_text
    }

# Function to print tables in a well-formatted manner
def print_tables(tables, table_type):
    """Print formatted tables."""
    print(f"\n{table_type} Tables:")
    for entry in tables:
        print(f"\nPage {entry['page']}:")
        if table_type == "Text-based":
            # Format table using tabulate for readability
            print(tabulate(entry['table'], headers="firstrow", tablefmt="grid"))
        else:
            # Print each line for image-based table OCR results
            for row in entry['table']:
                print(row)

# Function to print non-table text
def print_non_table_text(text_entries):
    """Print non-table text from PDF."""
    print("\nNon-table Text:")
    for entry in text_entries:
        print(f"\n--- Page {entry['page']} ---\n")
        print(entry['text'])

# Example usage
#pdf_path = "your_pdf_file.pdf"
#result = process_pdf(pdf_path)

# 1. Extract text-based tables using pdfplumber
print("Extracting text-based tables...")
result["text_based_tables"] = extract_text_based_tables_with_plumber(pdf_path)

# Print formatted results
print_tables(result["text_based_tables"], "Text-based")
#print_tables(result["image_based_tables"], "Image-based")

# 3. Extract non-table text using pdfplumber
print("\n\nExtracting non-table text...")
result["non_table_text"] = extract_non_table_text_with_plumber(pdf_path)

print_non_table_text(result["non_table_text"])


Extracting text-based tables...

Text-based Tables:

Page 1:
+------------------+----------------+----------------+
| T1Row 1, Col 1   | Row 1, Col 2   | Row 1, Col 3   |
+==================+================+================+
| T1Row 2, Col 1   | Row 2, Col 2   | Row 2, Col 3   |
+------------------+----------------+----------------+
| T1Row 3, Col 1   | Row 3, Col 2   | Row 3, Col 3   |
+------------------+----------------+----------------+

Page 2:
+------------------+----------------+----------------+
| T3Row 1, Col 1   | Row 1, Col 2   | Row 1, Col 3   |
+==================+================+================+
| T3Row 2, Col 1   | Row 2, Col 2   | Row 2, Col 3   |
+------------------+----------------+----------------+
| T3Row 3, Col 1   | Row 3, Col 2   | Row 3, Col 3   |
+------------------+----------------+----------------+


Extracting non-table text...
Error processing table on page 1: could not convert string to float: 'T1Row 1, Col 1'
Error processing table on page 2: could not

In [60]:
# 2. Extract image-based tables using EasyOCR
print("Extracting image-based tables...")
result["image_based_tables"] = extract_image_based_tables_with_easyocr(pdf_path)

print_tables(result["image_based_tables"], "Image-based")


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Extracting image-based tables...

Image-based Tables:

Page 1:
Sample Document with Tables and
Images
Table 1
TIRow 1
Col
Row 1,
Col 2
Row 1
Col 3
TIRow 2, Col
Row 2, Col 2
Row 2, Col 3
TIRow 3, Col
Row 3, Col 2
Row 3, Col 3
Some random text here Some random text here
Some random text here
Some random text here
Table 2
T2Row 1, Col 1
Row 1, Col2
Row 1
Col 3
T2Row 2, Col 1
Row 2, Col 2
Row 2, Col 3
T2Row 3,Col 1
Row 3, Col 2
Row 3,Col 3

Page 2:
Some random text here Some random text here
Table 3
T3Row 1
Col
Row 1,
Col 2
Row 12
Col 3
T3Row 2, Col
Row 2, Col 2
Row 2, Col 3
T3Row 3, Col 1
Row 3, Col 2
Row 3, Col 3

Page 3:
Page with no tables
Page with no tables Page with no tables
Page with no tables Page with no tables Page with no tables

Page 4:
Page 4
Image 1 with Tables and Text Inside
Description for Image 1:
[Placeholder for Image 1 showing tables and text]
Image1-Row 1,Col 1 Imagel-Row 1,Col 2
Imagel-Row 1, Col 3
Image1-Row2,Col 1 lmagez-Row 2, Col 2
Image1-Row 2, Col 3
Image1-Ro